# Part 2



In [2]:
import pandas as pd

xlsx_file = "DAMLZHBSPP_2023.xlsx"

# Create an ExcelFile object to inspect the file
xls = pd.ExcelFile(xlsx_file)

print(f"Sheets in {xlsx_file}: {xls.sheet_names}")

# Read the first sheet into a DataFrame for structural inspection
if xls.sheet_names:
    first_sheet_name = xls.sheet_names[0]
    df_first_sheet = pd.read_excel(xlsx_file, sheet_name=first_sheet_name)
    print(f"\nData structure of the first sheet ('{first_sheet_name}'):")
    display(df_first_sheet.head())
    df_first_sheet.info()
else:
    print("No sheets found in the Excel file.")

Sheets in DAMLZHBSPP_2023.xlsx: ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

Data structure of the first sheet ('Jan'):


,Delivery Date,Hour Ending,Repeated Hour Flag,Settlement Point,Settlement Point Price
0,01/01/2023,01:00,N,HB_BUSAVG,10.36
1,01/01/2023,01:00,N,HB_HOUSTON,10.37
2,01/01/2023,01:00,N,HB_HUBAVG,10.42
3,01/01/2023,01:00,N,HB_NORTH,10.48
4,01/01/2023,01:00,N,HB_PAN,7.88


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11160 entries, 0 to 11159
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Delivery Date           11160 non-null  object 
 1   Hour Ending             11160 non-null  object 
 2   Repeated Hour Flag      11160 non-null  object 
 3   Settlement Point        11160 non-null  object 
 4   Settlement Point Price  11160 non-null  float64
dtypes: float64(1), object(4)
memory usage: 436.1+ KB


In [3]:
import pandas as pd
from pathlib import Path
import numpy as np

xlsx = "DAMLZHBSPP_2023.xlsx"
zones = ["LZ_WEST", "LZ_HOUSTON"]

# Read all sheets and concatenate them row by row
xls = pd.ExcelFile(xlsx)
parts = []
for sh in xls.sheet_names:
    df = pd.read_excel(xlsx, sheet_name=sh)
    # Retain only the desired columns; column names should match the screenshot;
    # adjust if case/spacing differs.
    df = df[["Delivery Date", "Hour Ending", "Repeated Hour Flag", "Settlement Point", "Settlement Point Price"]]
    parts.append(df)
raw = pd.concat(parts, ignore_index=True)

# Retain only 'Load Zone' settlement points
raw = raw[raw["Settlement Point"].isin(zones)].copy()

# Combine to "local hourly timestamp" (HE=01 means the hour from 00:00-01:00, timestamp set at 01:00)
raw["Delivery Date"] = pd.to_datetime(raw["Delivery Date"])
# Hour Ending might be '01:00' or '1' etc., standardize to integer hour
he = pd.to_datetime(raw["Hour Ending"].astype(str), format="%H:%M", errors="coerce").dt.hour
he = he.fillna(pd.to_numeric(raw["Hour Ending"], errors="coerce"))  # Compatible with pure numbers
raw["HE_int"] = he.astype(pd.Int64Dtype()) # Modified to use nullable integer type
raw["datetime_local"] = raw["Delivery Date"] + pd.to_timedelta(raw["HE_int"], unit="h")

# Handle repeated hours due to fall daylight saving time rollback: if a 'datetime_local, Settlement Point' pair appears twice
# Simple approach: take the mean (could also keep the first entry)
raw = (
    raw
    .groupby(["Settlement Point", "datetime_local"], as_index=False)["Settlement Point Price"]
    .mean()
)

# Create the output directory if it doesn't exist
output_dir = Path("data_clean")
output_dir.mkdir(parents=True, exist_ok=True)

# Generate a two-column file for each load zone
for z in zones:
    sub = raw[raw["Settlement Point"] == z].copy()
    sub = sub.rename(columns={"Settlement Point Price": "price_usd_per_mwh"})
    sub = sub[["datetime_local", "price_usd_per_mwh"]].sort_values("datetime_local")

    # Optional: force alignment to 8760 hours (spring jump misses 1 hour -> interpolate; fall rollback already averaged above)
    # Construct full annual hourly index for 2023 (US Central Local Time)
    full = pd.date_range("2023-01-01 01:00", "2024-01-01 00:00", freq="h", tz=None, inclusive="left") # Changed 'H' to 'h'
    # Here, using "representing local time series" method, no timezone tag, to keep consistent with your PV CSV (both are local full hours)
    # If you need timezone tags, you can localize('America/Chicago') here

    sub = sub.set_index("datetime_local").reindex(full)
    sub.index.name = "datetime_local"
    # For missing values (spring daylight saving time jump), perform linear interpolation/forward fill
    sub["price_usd_per_mwh"] = sub["price_usd_per_mwh"].interpolate(limit_direction="both")

    out_path = output_dir / f"price_hourly_{z.split('_')[1]}_2023.csv"  # WEST/HOUSTON
    sub.to_csv(out_path, float_format="%.4f")
    print("saved:", out_path)

saved: data_clean/price_hourly_WEST_2023.csv
saved: data_clean/price_hourly_HOUSTON_2023.csv


In [4]:
import pandas as pd
from pathlib import Path
import numpy as np

RTM = "DAMLZHBSPP_2023.xlsx"
zones = ["LZ_WEST", "LZ_HOUSTON"]

# Read all sheets and concatenate them row by row
xls = pd.ExcelFile(RTM)
parts = []
for sh in xls.sheet_names:
    df = pd.read_excel(RTM, sheet_name=sh)
    # Retain only the desired columns; column names should match the screenshot;
    # adjust if case/spacing differs.
    df = df[["Delivery Date", "Hour Ending", "Repeated Hour Flag", "Settlement Point", "Settlement Point Price"]]
    parts.append(df)
raw = pd.concat(parts, ignore_index=True)

# Retain only 'Load Zone' settlement points
raw = raw[raw["Settlement Point"].isin(zones)].copy()

# Combine to "local hourly timestamp" (HE=01 means the hour from 00:00-01:00, timestamp set at 01:00)
raw["Delivery Date"] = pd.to_datetime(raw["Delivery Date"])
# Hour Ending might be '01:00' or '1' etc., standardize to integer hour
he = pd.to_datetime(raw["Hour Ending"].astype(str), format="%H:%M", errors="coerce").dt.hour
he = he.fillna(pd.to_numeric(raw["Hour Ending"], errors="coerce"))  # Compatible with pure numbers
raw["HE_int"] = he.astype(pd.Int64Dtype()) # Modified to use nullable integer type
raw["datetime_local"] = raw["Delivery Date"] + pd.to_timedelta(raw["HE_int"], unit="h")

# Handle repeated hours due to fall daylight saving time rollback: if a 'datetime_local, Settlement Point' pair appears twice
# Simple approach: take the mean (could also keep the first entry)
raw = (
    raw
    .groupby(["Settlement Point", "datetime_local"], as_index=False)["Settlement Point Price"]
    .mean()
)

# Create the output directory if it doesn't exist
output_dir = Path("data_clean")
output_dir.mkdir(parents=True, exist_ok=True)

# Generate a two-column file for each load zone
for z in zones:
    sub = raw[raw["Settlement Point"] == z].copy()
    sub = sub.rename(columns={"Settlement Point Price": "price_usd_per_mwh"})
    sub = sub[["datetime_local", "price_usd_per_mwh"]].sort_values("datetime_local")

    # Optional: force alignment to 8760 hours (spring jump misses 1 hour -> interpolate; fall rollback already averaged above)
    # Construct full annual hourly index for 2023 (US Central Local Time)
    full = pd.date_range("2023-01-01 01:00", "2024-01-01 00:00", freq="h", tz=None, inclusive="left") # Changed 'H' to 'h'
    # Here, using "representing local time series" method, no timezone tag, to keep consistent with your PV CSV (both are local full hours)
    # If you need timezone tags, you can localize('America/Chicago') here

    sub = sub.set_index("datetime_local").reindex(full)
    sub.index.name = "datetime_local"
    # For missing values (spring daylight saving time jump), perform linear interpolation/forward fill
    sub["price_usd_per_mwh"] = sub["price_usd_per_mwh"].interpolate(limit_direction="both")

    out_path = output_dir / f"RTM_{z.split('_')[1]}_2023.csv"  # WEST/HOUSTON
    sub.to_csv(out_path, float_format="%.4f")
    print("saved:", out_path)

saved: data_clean/RTM_WEST_2023.csv
saved: data_clean/RTM_HOUSTON_2023.csv


In [ ]:
import pandas as pd
from pathlib import Path

def build_hourly_merged(zone: str, year: int):
    zone = zone.upper()
    in_pv   = Path(f"data_clean/pv_hourly_{zone}_{year}.csv")
    in_px   = Path(f"data_clean/price_hourly_{zone}_{year}.csv")
    out_dir = Path("out"); out_dir.mkdir(parents=True, exist_ok=True)
    out_fp  = out_dir / f"hourly_merged_{zone}_{year}.parquet"

    # 1) Read and standardize
    pv = pd.read_csv(in_pv)
    px = pd.read_csv(in_px)

    # Standardize column names (to prevent case or space differences)
    pv = pv.rename(columns={"Datetime_local":"datetime_local", "DateTime_local":"datetime_local"})
    px = px.rename(columns={"Datetime_local":"datetime_local", "DateTime_local":"datetime_local",
                            "price":"price_usd_per_mwh", "Price":"price_usd_per_mwh"})

    pv["datetime_local"] = pd.to_datetime(pv["datetime_local"])
    px["datetime_local"] = pd.to_datetime(px["datetime_local"])

    # 2) Basic checks (can print logs)
    # Remove duplicates, to prevent duplicate entries for the same hour
    pv = pv.drop_duplicates(subset=["datetime_local"]).sort_values("datetime_local")
    px = px.drop_duplicates(subset=["datetime_local"]).sort_values("datetime_local")

    # 3) Merge (inner join, only hours present in both sides are retained)
    df = pv.merge(px, on="datetime_local", how="inner")

    # 4) Calculate revenue
    # Unit: ac_mw( MW ) × price_usd_per_mwh( $/MWh ) → $/h
    df["solar_revenue"] = df["ac_mw"] * df["price_usd_per_mwh"]

    # 5) Select and sort columns
    df = df[["datetime_local", "ac_mw", "price_usd_per_mwh", "solar_revenue"]].sort_values("datetime_local")

    # 6) QA (simple assertions/hints)
    # Number of rows (ideally 8760; if the price table previously kept 8759/8784, it will also end up here)
    n = len(df)
    print(f"[{zone} {year}] rows={n} (expected ~8760)")
    # Missing values
    na = df.isna().sum()
    if na.any():
        print("NaN counts:\n", na)
    # Negative prices/negative revenue (can exist, only as a hint)
    neg_p = (df["price_usd_per_mwh"] < 0).sum()
    neg_r = (df["solar_revenue"] < 0).sum()
    print(f"negative prices={neg_p}, negative revenue hours={neg_r}")

    # 7) Write Parquet (faster and smaller)
    df.to_parquet(out_fp, index=False)
    print("saved ->", out_fp)

# Example: Batch processing for two zones
for z in ["WEST", "HOUSTON"]:
    build_hourly_merged(z, 2023)

In [7]:
import pandas as pd

# Define the path to the CSV file
west_pv_file = "/content/WEST pvwatts_hourly_cleaned.csv"

# Load the CSV file into a pandas DataFrame
df_west_pv = pd.read_csv(west_pv_file)

print(f"Data structure of '{west_pv_file}':\n")
# Display the first 5 rows of the DataFrame
display(df_west_pv.head())

# Display information about the DataFrame, including data types and non-null counts
df_west_pv.info()

Data structure of '/content/WEST pvwatts_hourly_cleaned.csv':



,Month,Day,Hour,AC System Output (W),AC System Output (MW)
0,1,1,0,0.0,0.0
1,1,1,1,0.0,0.0
2,1,1,2,0.0,0.0
3,1,1,3,0.0,0.0
4,1,1,4,0.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Month                  8760 non-null   int64  
 1   Day                    8760 non-null   int64  
 2   Hour                   8760 non-null   int64  
 3   AC System Output (W)   8760 non-null   float64
 4   AC System Output (MW)  8760 non-null   float64
dtypes: float64(2), int64(3)
memory usage: 342.3 KB


In [5]:
import pandas as pd

# Define the path to the CSV file
houston_pv_file = "/content/HUSTON pvwatts_hourly_cleaned.csv"

# Load the CSV file into a pandas DataFrame
df_houston_pv = pd.read_csv(houston_pv_file)

print(f"Data structure of '{houston_pv_file}':\n")
# Display the first 5 rows of the DataFrame
display(df_houston_pv.head())

# Display information about the DataFrame, including data types and non-null counts
df_houston_pv.info()

Data structure of '/content/HUSTON pvwatts_hourly_cleaned.csv':



,Month,Day,Hour,AC System Output (W),AC System Output (MW)
0,1,1,0,0.0,0.0
1,1,1,1,0.0,0.0
2,1,1,2,0.0,0.0
3,1,1,3,0.0,0.0
4,1,1,4,0.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Month                  8760 non-null   int64  
 1   Day                    8760 non-null   int64  
 2   Hour                   8760 non-null   int64  
 3   AC System Output (W)   8760 non-null   float64
 4   AC System Output (MW)  8760 non-null   float64
dtypes: float64(2), int64(3)
memory usage: 342.3 KB


## Part 3

In [8]:
import pandas as pd
from pathlib import Path

YEAR = 2023

# ① PV files
PV_FILES = {
    "WEST":    "WEST pvwatts_hourly_cleaned.csv",
    "HOUSTON": "HUSTON pvwatts_hourly_cleaned.csv",
}

# ② Price files (DAM baseline)
def price_path(zone: str, year: int) -> Path:
    return Path(f"data_clean/price_hourly_{zone}_{year}.csv")

# -- Utility: Convert PVWatts clean table to standard two columns -- #
def load_pv_as_hourly(pv_csv: str, year: int) -> pd.DataFrame:
    """
    Input: Hourly table exported from PVWatts/SAM (contains Month, Day, Hour, AC System Output (W)/(MW))
    Output: Two columns (datetime_local, ac_mw)
    """
    pv = pd.read_csv(pv_csv)

    # Column name robustness (remove spaces/case-insensitivity)
    colmap = {c.strip().lower(): c for c in pv.columns}
    month = colmap.get("month")
    day   = colmap.get("day")
    hour  = colmap.get("hour")

    # Select MW column; if no MW column, use W / 1e6
    ac_mw_col = colmap.get("ac system output (mw)")
    ac_w_col  = colmap.get("ac system output (w)")

    if ac_mw_col:
        pv["ac_mw"] = pv[ac_mw_col].astype(float)
    elif ac_w_col:
        pv["ac_mw"] = pv[ac_w_col].astype(float) / 1e6
    else:
        raise ValueError("AC column not found: requires 'AC System Output (MW)' or 'AC System Output (W)'")

    # Construct local hourly time (aligned with price convention; Hour=0..23)
    pv["datetime_local"] = pd.to_datetime(
        dict(year=year, month=pv[month], day=pv[day], hour=pv[hour]),
        errors="coerce"
    )

    out = (pv[["datetime_local", "ac_mw"]]
           .dropna(subset=["datetime_local"])
           .drop_duplicates(subset=["datetime_local"])
           .sort_values("datetime_local"))
    return out

def build_hourly_merged(zone: str, year: int):
    zone = zone.upper()
    pv_path = PV_FILES[zone]
    px_path = price_path(zone, year)

    # Output path
    out_dir = Path("out"); out_dir.mkdir(parents=True, exist_ok=True)
    out_fp  = out_dir / f"hourly_merged_{zone}_{year}.parquet"

    # 1) Read PV (construct datetime_local/ac_mw)
    pv = load_pv_as_hourly(pv_path, year)

    # 2) Read prices (two columns: datetime_local, price_usd_per_mwh)
    px = pd.read_csv(px_path)
    px = px.rename(columns={
        "Datetime_local": "datetime_local",
        "DateTime_local": "datetime_local",
        "Settlement Point Price": "price_usd_per_mwh",
        "price": "price_usd_per_mwh",
        "Price": "price_usd_per_mwh",
    })
    px["datetime_local"] = pd.to_datetime(px["datetime_local"])
    px = px[["datetime_local", "price_usd_per_mwh"]].drop_duplicates("datetime_local")

    # 3) Merge
    df = pv.merge(px, on="datetime_local", how="inner")

    # 4) Calculate hourly revenue
    df["solar_revenue"] = df["ac_mw"] * df["price_usd_per_mwh"]

    # 5) Select columns and sort
    df = df[["datetime_local", "ac_mw", "price_usd_per_mwh", "solar_revenue"]].sort_values("datetime_local")

    # 6) QA
    print(f"[{zone} {year}] rows={len(df)}  "
          f"neg_price={ (df.price_usd_per_mwh<0).sum() }  "
          f"neg_rev={ (df.solar_revenue<0).sum() }  "
          f"ac_sum_mwh≈{df['ac_mw'].sum():.1f}")

    # 7) Write Parquet
    df.to_parquet(out_fp, index=False)
    print("saved ->", out_fp)

# Process two zones at once (3.1)
for z in ["WEST", "HOUSTON"]:
    build_hourly_merged(z, YEAR)

[WEST 2023] rows=8759  neg_price=100  neg_rev=87  ac_sum_mwh≈211792.3
saved -> out/hourly_merged_WEST_2023.parquet
[HOUSTON 2023] rows=8759  neg_price=0  neg_rev=0  ac_sum_mwh≈171300.8
saved -> out/hourly_merged_HOUSTON_2023.parquet


In [9]:
import pandas as pd
from pathlib import Path

output_dir = Path("out")

# List of zones to process
zones = ["WEST", "HOUSTON"]

for z in zones:
    # Define input and output paths
    parquet_file = output_dir / f"hourly_merged_{z}_2023.parquet"
    csv_file = output_dir / f"hourly_merged_{z}_2023.csv"

    # Read the parquet file into a DataFrame
    df = pd.read_parquet(parquet_file)

    # Save the DataFrame to a CSV file
    df.to_csv(csv_file, index=False, float_format="%.4f")

    print(f"Converted '{parquet_file}' to '{csv_file}'")


Converted 'out/hourly_merged_WEST_2023.parquet' to 'out/hourly_merged_WEST_2023.csv'
Converted 'out/hourly_merged_HOUSTON_2023.parquet' to 'out/hourly_merged_HOUSTON_2023.csv'


In [10]:
import pandas as pd
from pathlib import Path

def _read_merged(zone: str, year: int) -> pd.DataFrame:
    """Prioritize reading CSV; if not found, read Parquet. Requires datetime_local, price_usd_per_mwh"""
    zone = zone.upper()
    p_parq = Path(f"out/hourly_merged_{zone}_{year}.parquet")
    p_csv  = Path(f"out/hourly_merged_{zone}_{year}.csv")
    if p_csv.exists():
        df = pd.read_csv(p_csv)
    elif p_parq.exists():
        df = pd.read_parquet(p_parq)
    else:
        raise FileNotFoundError(f"Merged file from 3.1 not found: {p_parq} or {p_csv}")
    df["datetime_local"] = pd.to_datetime(df["datetime_local"])
    df = df.sort_values("datetime_local").drop_duplicates("datetime_local")
    # Only need price column (dispatch depends on price; PV energy does not affect battery arbitrage logic)
    need = ["datetime_local","price_usd_per_mwh"]
    missing = [c for c in need if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}; please check 3.1 output")
    return df[need].copy()

def battery_dispatch_daily_extrema(df: pd.DataFrame,
                                   power_mw: float,
                                   energy_mwh: float,
                                   rt_efficiency: float = 0.9) -> pd.DataFrame:
    """
    Simplified daily arbitrage: each day, select k hours with lowest price to charge, k hours with highest price to discharge (k=energy/power).
    Simulate SOC sequentially by time, ensuring 0 ≤ SOC ≤ Emax and power limits; maximum ~1 cycle/day.
    """
    k = int(round(energy_mwh / power_mw))  # Number of charge/discharge hours required
    if k < 1:
        raise ValueError("energy_mwh / power_mw < 1 ? Please check parameters.")

    df = df.copy()
    df["charge_mw"] = 0.0
    df["discharge_mw"] = 0.0

    # Mark charge/discharge sets for each day (sets may overlap in time; feasibility handled by sequential SOC simulation later)
    g = df.groupby(df["datetime_local"].dt.date, sort=True)
    for day, idx in g.groups.items():
        sub = df.loc[idx]

        # k hours lowest price -> tendency to charge; k hours highest price -> tendency to discharge
        low_idx  = sub.nsmallest(k, "price_usd_per_mwh").index
        high_idx = sub.nlargest(k, "price_usd_per_mwh").index

        df.loc[low_idx,  "charge_mw"]    = power_mw
        df.loc[high_idx, "discharge_mw"] = power_mw

        # If the same hour is selected for both, prioritize 'charge'; discharge will be determined by SOC during simulation


    # Simulate SOC sequentially by time (charge: SOC += charge_mw; discharge: SOC -= discharge_mw)
    soc = 0.0
    Emax = energy_mwh
    ch = []
    dis = []
    soc_list = []
    profit = []

    for _, row in df.iterrows():
        price = row["price_usd_per_mwh"]
        want_charge = row["charge_mw"]
        want_dis    = row["discharge_mw"]

        # First, determine feasible output for charge/discharge in this hour (cannot charge and discharge simultaneously)
        charge = 0.0
        disch  = 0.0

        if want_charge > 0 and want_dis == 0:
            # Limited by SOC upper limit; charged energy = charge (MWh/h=MW), but electricity purchase is reduced by efficiency
            room = Emax - soc
            charge = min(want_charge, room)  # MW
            soc += charge
        elif want_dis > 0 and want_charge == 0:
            # Limited by SOC lower limit; discharged energy = disch (MWh/h=MW)
            available = soc
            disch = min(want_dis, available)
            soc -= disch
        else:
            # If both charge/discharge or neither are marked -> no action
            charge = 0.0
            disch  = 0.0

        # Hourly profit: revenue from selling electricity - cost of buying electricity (purchase needs to be divided by efficiency)
        hour_profit = price * disch - price * (charge / rt_efficiency)

        ch.append(charge)
        dis.append(disch)
        soc_list.append(soc)
        profit.append(hour_profit)

    df["charge_mw"]       = ch
    df["discharge_mw"]    = dis
    df["soc_mwh"]         = soc_list
    df["batt_profit_hour"] = profit

    # Approximate cumulative cycles (cumulative discharged energy / Emax)
    discharged_energy = df["discharge_mw"].sum()  # MW·h = MWh
    df["cycles_cum"] = discharged_energy / Emax

    return df

def run_dispatch(zone: str, year: int,
                 power_mw: float = 100.0,
                 energy_mwh: float = 400.0,
                 rt_efficiency: float = 0.9):
    zone = zone.upper()
    dur_h = int(round(energy_mwh / power_mw))
    base = _read_merged(zone, year)  # Contains datetime_local, price_usd_per_mwh

    out = battery_dispatch_daily_extrema(base, power_mw, energy_mwh, rt_efficiency)

    out_dir = Path("out"); out_dir.mkdir(exist_ok=True, parents=True)
    p_parq = out_dir / f"battery_dispatch_{zone}_{year}_{dur_h}h.parquet"
    p_csv  = out_dir / f"battery_dispatch_{zone}_{year}_{dur_h}h.csv"

    out.to_parquet(p_parq, index=False)
    out.to_csv(p_csv, index=False, float_format="%.4f", date_format="%Y-%m-%d %H:%M:%S")

    print(f"[{zone}] {year}  power={power_mw}MW  energy={energy_mwh}MWh  η={rt_efficiency}")
    print(f"profit_year = ${out['batt_profit_hour'].sum():,.0f}")
    print(f"cycles≈ {out['cycles_cum'].iloc[-1]:.1f}")
    print("saved ->", p_parq, "and", p_csv)

# Run for two zones (example: 100MW/400MWh, η=0.9)
for Z in ["WEST", "HOUSTON"]:
    run_dispatch(Z, 2023, power_mw=100, energy_mwh=400, rt_efficiency=0.9)

[WEST] 2023  power=100MW  energy=400MWh  η=0.9
profit_year = $25,101,514
cycles≈ 323.0
saved -> out/battery_dispatch_WEST_2023_4h.parquet and out/battery_dispatch_WEST_2023_4h.csv
[HOUSTON] 2023  power=100MW  energy=400MWh  η=0.9
profit_year = $23,478,747
cycles≈ 342.2
saved -> out/battery_dispatch_HOUSTON_2023_4h.parquet and out/battery_dispatch_HOUSTON_2023_4h.csv


In [11]:
import pandas as pd
from pathlib import Path

YEAR   = 2023
ZONES  = ["WEST", "HOUSTON"]
DUR_H  = 4                       # Battery duration (hours), must match 3.2 output (e.g., 4h)

def _read_any(path_parquet: Path, path_csv: Path) -> pd.DataFrame:
    if path_parquet.exists():
        return pd.read_parquet(path_parquet)
    if path_csv.exists():
        return pd.read_csv(path_csv)
    raise FileNotFoundError(f"not found: {path_parquet} or {path_csv}")

def build_sps_and_annual(zone: str, year: int, dur_h: int):
    zone = zone.upper()
    out_dir = Path("out"); out_dir.mkdir(parents=True, exist_ok=True)

    # --- Input files ---
    p_merged_parq = out_dir / f"hourly_merged_{zone}_{year}.parquet"
    p_merged_csv  = out_dir / f"hourly_merged_{zone}_{year}.csv"

    p_batt_parq   = out_dir / f"battery_dispatch_{zone}_{year}_{dur_h}h.parquet"
    p_batt_csv    = out_dir / f"battery_dispatch_{zone}_{year}_{dur_h}h.csv"

    # Read 3.1 (PV + Price hourly table)
    merged = _read_any(p_merged_parq, p_merged_csv).copy()
    merged["datetime_local"] = pd.to_datetime(merged["datetime_local"])
    merged = merged.drop_duplicates("datetime_local").sort_values("datetime_local")

    # Read 3.2 (Battery dispatch hourly table)
    batt = _read_any(p_batt_parq, p_batt_csv).copy()
    batt["datetime_local"] = pd.to_datetime(batt["datetime_local"])
    batt = batt.drop_duplicates("datetime_local").sort_values("datetime_local")

    # Compatibility: if 3.1 did not calculate revenue_pv, calculate it here
    if "revenue_pv" not in merged.columns:
        if {"ac_mw","price_usd_per_mwh"}.issubset(merged.columns):
            merged["revenue_pv"] = merged["ac_mw"] * merged["price_usd_per_mwh"]
        else:
            raise ValueError("hourly_merged is missing revenue_pv and cannot be calculated from ac_mw*price; please check 3.1 output.")

    # --- Merge into S+S hourly table ---
    sps = merged.merge(
        batt[["datetime_local","charge_mw","discharge_mw","soc_mwh","batt_profit_hour"]],
        on="datetime_local", how="left"
    )
    sps["charge_mw"] = sps["charge_mw"].fillna(0.0)
    sps["discharge_mw"] = sps["discharge_mw"].fillna(0.0)
    sps["batt_profit_hour"] = sps["batt_profit_hour"].fillna(0.0)

    sps["net_batt_mw"] = sps["discharge_mw"] - sps["charge_mw"]
    sps["revenue_total_hour"] = sps["revenue_pv"] + sps["batt_profit_hour"]

    # Output hourly S+S
    p_sps_parq = out_dir / f"hourly_solar_plus_storage_{zone}_{year}_{dur_h}h.parquet"
    p_sps_csv  = out_dir / f"hourly_solar_plus_storage_{zone}_{year}_{dur_h}h.csv"
    sps.to_parquet(p_sps_parq, index=False)
    sps.to_csv(p_sps_csv, index=False, float_format="%.4f", date_format="%Y-%m-%d %H:%M:%S")
    print("saved ->", p_sps_parq, "and", p_sps_csv)

    # --- Annual Summary (for 4.x) ---
    # PV
    pv_annual = {
        "tech": "Solar_PV",
        "zone": zone,
        "year": year,
        "annual_mwh": merged["ac_mw"].sum(),                     # Hourly MW accumulated ≈ Annual MWh
        "annual_revenue_usd": merged["revenue_pv"].sum(),
        "annual_batt_profit_usd": 0.0,
        "market": "DAM",  # If you know it's DAM or RTM, you can write it in 3.1 and use it
    }
    # S+S
    sps_annual = {
        "tech": f"Solar_PV_Storage_{dur_h}h",
        "zone": zone,
        "year": year,
        "annual_mwh": merged["ac_mw"].sum(),                     # Generation same as PV; battery only changes revenue
        "annual_revenue_usd": sps["revenue_total_hour"].sum(),
        "annual_batt_profit_usd": sps["batt_profit_hour"].sum(),
        "market": "DAM",
    }

    annual_rows = pd.DataFrame([pv_annual, sps_annual])

    # Append/create out/annual_ops_summary.csv
    p_summary = out_dir / "annual_ops_summary.csv"
    if p_summary.exists():
        old = pd.read_csv(p_summary)
        all_df = pd.concat([old, annual_rows], ignore_index=True)
    else:
        all_df = annual_rows
    all_df.to_csv(p_summary, index=False)
    print("updated ->", p_summary)

    # Small QA
    print(f"[{zone}] PV revenue = ${pv_annual['annual_revenue_usd']:,.0f}  "
          f"S+S ({dur_h}h) revenue = ${sps_annual['annual_revenue_usd']:,.0f}  "
          f"(battery profit = ${sps_annual['annual_batt_profit_usd']:,.0f})")

# Run for two zones at once
for z in ZONES:
    build_sps_and_annual(z, YEAR, DUR_H)

saved -> out/hourly_solar_plus_storage_WEST_2023_4h.parquet and out/hourly_solar_plus_storage_WEST_2023_4h.csv
updated -> out/annual_ops_summary.csv
[WEST] PV revenue = $8,704,718  S+S (4h) revenue = $33,806,232  (battery profit = $25,101,514)
saved -> out/hourly_solar_plus_storage_HOUSTON_2023_4h.parquet and out/hourly_solar_plus_storage_HOUSTON_2023_4h.csv
updated -> out/annual_ops_summary.csv
[HOUSTON] PV revenue = $7,002,626  S+S (4h) revenue = $30,481,373  (battery profit = $23,478,747)


# Part 4

In [12]:
import pandas as pd
from pathlib import Path
import numpy as np
from glob import glob

OUT = Path("out"); OUT.mkdir(exist_ok=True, parents=True)

def _read_any(parq_or_csv):
    p = Path(parq_or_csv)
    if p.suffix == ".parquet":
        return pd.read_parquet(p)
    else:
        return pd.read_csv(p)

def rebuild_annual_ops_summary(year: int, zones=("WEST","HOUSTON"), market="DAM"):
    rows = []

    for zone in zones:
        zone = zone.upper()
        # 3.1: PV + Prices (required)
        p_merged = []
        for ext in ("parquet","csv"):
            cand = OUT / f"hourly_merged_{zone}_{year}.{ext}"
            if cand.exists(): p_merged.append(str(cand))
        if not p_merged:
            raise FileNotFoundError(f"missing 3.1 for {zone}: hourly_merged_{zone}_{year}.[parquet|csv]")

        merged = _read_any(p_merged[0]).copy()
        merged["datetime_local"] = pd.to_datetime(merged["datetime_local"])
        merged = merged.drop_duplicates("datetime_local").sort_values("datetime_local")

        # If 3.1 didn't include revenue_pv, calculate it here
        if "revenue_pv" not in merged.columns:
            if {"ac_mw","price_usd_per_mwh"}.issubset(merged.columns):
                merged["revenue_pv"] = merged["ac_mw"] * merged["price_usd_per_mwh"]
            else:
                raise ValueError("hourly_merged is missing revenue_pv and cannot be calculated.")

        # Price diagnostics
        neg_price_hours = int((merged["price_usd_per_mwh"] < 0).sum())
        mean_price = float(merged["price_usd_per_mwh"].mean())
        p95_price = float(merged["price_usd_per_mwh"].quantile(0.95))

        # --- PV Annual Summary ---
        rows.append({
            "tech": "Solar_PV",
            "zone": zone,
            "year": year,
            "market": market,
            "annual_mwh": float(merged["ac_mw"].sum()),
            "annual_revenue_usd": float(merged["revenue_pv"].sum()),
            "annual_batt_profit_usd": 0.0,
            "negative_price_hours": neg_price_hours,
            "mean_price": round(mean_price, 3),
            "p95_price": round(p95_price, 3),
        })

        # 3.2: Battery (optional, if present, summarize S+S)
        # Automatically match any duration: battery_dispatch_{zone}_{year}_*h.[parquet|csv]
        patt = str(OUT / f"battery_dispatch_{zone}_{year}_*h.*")
        for fp in glob(patt):
            batt = _read_any(fp).copy()
            batt["datetime_local"] = pd.to_datetime(batt["datetime_local"])
            batt = batt.drop_duplicates("datetime_local").sort_values("datetime_local")

            dur = fp.split("_")[-1].split("h")[0]  # Roughly extract duration
            batt_profit = float(batt["batt_profit_hour"].sum())

            # S+S: Use merged PV generation + battery arbitrage profit
            rows.append({
                "tech": f"Solar_PV_Storage_{dur}h",
                "zone": zone,
                "year": year,
                "market": market,
                "annual_mwh": float(merged["ac_mw"].sum()),
                "annual_revenue_usd": float(merged["revenue_pv"].sum() + batt_profit),
                "annual_batt_profit_usd": batt_profit,
                "negative_price_hours": neg_price_hours,
                "mean_price": round(mean_price, 3),
                "p95_price": round(p95_price, 3),
            })

    df = pd.DataFrame(rows)
    df = df.sort_values(["zone","tech"])
    df.to_csv(OUT / "annual_ops_summary.csv", index=False)
    print("saved -> out/annual_ops_summary.csv")
    return df

rebuild_annual_ops_summary(2023, zones=("WEST","HOUSTON"), market="DAM")

saved -> out/annual_ops_summary.csv


,tech,zone,year,market,annual_mwh,annual_revenue_usd,annual_batt_profit_usd,negative_price_hours,mean_price,p95_price
3,Solar_PV,HOUSTON,2023,DAM,171300.799259,7.002626e+06,0.000000e+00,0,57.560,136.242
4,Solar_PV_Storage_4h,HOUSTON,2023,DAM,171300.799259,3.048137e+07,2.347875e+07,0,57.560,136.242
5,Solar_PV_Storage_4h,HOUSTON,2023,DAM,171300.799259,3.048137e+07,2.347875e+07,0,57.560,136.242
0,Solar_PV,WEST,2023,DAM,211792.276475,8.704718e+06,0.000000e+00,100,64.752,150.867
1,Solar_PV_Storage_4h,WEST,2023,DAM,211792.276475,3.380623e+07,2.510151e+07,100,64.752,150.867
2,Solar_PV_Storage_4h,WEST,2023,DAM,211792.276475,3.380623e+07,2.510151e+07,100,64.752,150.867


In [22]:
import pandas as pd
import numpy as np
from pathlib import Path

Path("out").mkdir(parents=True, exist_ok=True)

# Read inputs
tech = pd.read_csv("1.1 tech_params.csv", encoding='latin1')
policy = pd.read_excel("1.2 policy_params_from_epa.xlsx")  # Select one row for baseline
ops = pd.read_csv("out/annual_ops_summary.csv")   # Generated in 3.3

# Get baseline policy (can loop for multiple scenarios)
pol = policy.iloc[0].to_dict()
carbon_price = pol.get("carbon_price_usd_per_t", 0.0)
co2_ccgt     = pol.get("co2_t_per_mwh_ccgt", 0.36)  # Example value; refer to your 1.2 table
itc_percent  = pol.get("itc_percent", 0.0)
ptc          = pol.get("ptc_usd_per_mwh", 0.0)

def annuity_factor(r, n):
    return (r*(1+r)**n)/(((1+r)**n)-1)

rows = []
for (_, o) in ops.iterrows():
    zone  = o["zone"]
    year  = int(o["year"])
    tech_name = o["tech"]  # E.g., Solar_PV or Solar_PV_Storage_4h
    annual_mwh = float(o["annual_mwh"])
    batt_profit = float(o["annual_batt_profit_usd"])  # 0 for PV, battery arbitrage for S+S

    # Match tech_params (prefix match: Solar_PV_Storage_4h also falls under Solar_PV_Storage)
    if tech_name.startswith("Solar_PV_Storage"):
        trow = tech[tech["tech"]=="Solar_PV_Storage"].iloc[0].to_dict()
    else:
        trow = tech[tech["tech"]==tech_name].iloc[0].to_dict()

    capex = trow["capex_per_kw"]          # $/kW
    fom   = trow["fom_per_kw_yr"]         # $/kW-yr
    vom   = trow["vom_per_mwh"]           # $/MWh
    nlife = int(trow["lifetime_years"])
    wacc  = float(trow.get("wacc", trow.get("wacc_base", 0.08)))

    # Annualized capex: Note ITC (if used) can directly offset Capex × ITC% in a simplified approach
    capex_eff = capex * (1 - itc_percent/100.0)
    AF = annuity_factor(wacc, nlife)
    annualized_capex_per_kw_yr = capex_eff * AF  # $/kW-yr

    # Convert to $/MWh: Annual MWh / Installed kW = Annual equivalent full load hours ≈ 8760 * CF
    # Use "annual generation per kW" = annual_mwh / capacity_kw
    # Capacity is missing here, simplified: using classic LCOE formula: (Capex*AF + FOM)/AnnualMwh + VOM (+ fuel + carbon)
    # So, we first calculate (Capex*AF + FOM) as $/kW-yr, then divide by "annual generation per kW". Need annual MWh per kW:
    # We can infer capacity from annual_mwh and assumed installed capacity (more rigorously from inputs/capacity_config.csv).
    # Temporary simplification: Provide a mapping (according to your capacity_config in MW):
    capacity_map = {
        ("Solar_PV", zone, year): 100.0,      # MWac
        ("Solar_PV_Storage_4h", zone, year): 100.0,  # PV generation part is still 100MWac
        # If there are CCGT annual rows, fill them here, e.g., 500.0
    }

    cap_mw = capacity_map.get((tech_name, zone, year), np.nan)
    if np.isnan(cap_mw):
        # If no mapping, skip (or read from capacity_config yourself)
        continue
    annual_mwh_per_kw = annual_mwh / (cap_mw*1000.0)

    lcoe_core = (annualized_capex_per_kw_yr + fom) / (annual_mwh_per_kw + 1e-9) + vom

    # Fuel + carbon for CCGT (only add if tech==CCGT)
    fuel = 0.0
    carbon = 0.0
    if tech_name == "CCGT":
        gas_price = pol.get("gas_price_usd_per_mmbtu", 4.0)   # If policy has this column, use it
        heat_rate = trow.get("heat_rate_mmbtu_per_mwh", 7.0)  # From tech_params
        fuel = gas_price * heat_rate
        carbon = carbon_price * co2_ccgt

    # Battery arbitrage credit: annual arbitrage divided by annual MWh (only for S+S)
    credit = 0.0
    if tech_name.startswith("Solar_PV_Storage"):
        credit = batt_profit / (annual_mwh + 1e-9)

    lcoe = lcoe_core + fuel + carbon - credit - ptc  # If PTC ($/MWh) exists, subtract directly

    rows.append({
        "tech": tech_name,
        "zone": zone,
        "year": year,
        "lcoe_usd_per_mwh": round(lcoe, 2),
        "components": round(lcoe_core, 2),
        "fuel": round(fuel, 2),
        "carbon": round(carbon, 2),
        "storage_credit": round(credit, 2),
        "ptc": round(ptc, 2),
    })

lcoe_df = pd.DataFrame(rows)
lcoe_df.to_csv("out/lcoe_table.csv", index=False)
print("saved -> out/lcoe_table.csv")

saved -> out/lcoe_table.csv


In [25]:
import pandas as pd

# Load the IRR/NPV summary table
irr_npv_summary_df = pd.read_csv("out/irr_npv_table.csv")

print("Summary of calculated IRR and NPV values:")
display(irr_npv_summary_df)

Summary of calculated IRR and NPV values:


,tech,zone,year,npv_usd,irr_pct,payback_year
0,Solar_PV,HOUSTON,2023,-81019866.0,-1.64,23.9
1,Solar_PV_Storage_4h,HOUSTON,2023,149497931.0,20.77,4.7
2,Solar_PV_Storage_4h,HOUSTON,2023,149497931.0,20.77,4.7
3,Solar_PV,WEST,2023,-64308479.0,0.78,18.4
4,Solar_PV_Storage_4h,WEST,2023,182141889.0,23.32,4.2
5,Solar_PV_Storage_4h,WEST,2023,182141889.0,23.32,4.2


In [27]:
import pandas as pd
import numpy as np
from pathlib import Path
from glob import glob
import re

# ================== Path Configuration ==================
TECH_PATH = "/content/1.1 tech_params.csv"
POLI_PATH = "/content/1.2 policy_params_from_epa.xlsx"
OPS_PATH  = "out/annual_ops_summary.csv"   # Generated in 3.3
OUT_DIR   = Path("out")
OUT_DIR.mkdir(parents=True, exist_ok=True)
# ========================================================

INCLUDE_BATTERY_REPLACEMENT = False
INCLUDE_THROUGHPUT_COST     = False

# Battery replacement parameters (only effective if INCLUDE_BATTERY_REPLACEMENT=True)
battery_replacement_year   = 12          # Common 10–12
battery_replacement_ratio  = 0.7         # Replacement cost ≈ ratio of initial battery CAPEX (0.6–0.8 common)

# Battery throughput cost parameters (only effective if INCLUDE_THROUGHPUT_COST=True)
throughput_cost_per_mwh    = 8.0         # $/MWh_discharge (5–15 common)

def read_table_robust(path: str):
    try:
        if path.lower().endswith((".xlsx", ".xls")):
            df = pd.read_excel(path)
        else:
            df = None
            for enc in ("utf-8", "utf-8-sig", "latin1", "cp1252", "utf-16", "utf-16le", "utf-16be"):
                try:
                    df = pd.read_csv(path, encoding=enc)
                    break
                except Exception:
                    continue
            if df is None:

                df = pd.read_excel(path)
    except Exception as e:
        raise RuntimeError(f"Failed to read: {path}\n{e}")

    df.columns = [re.sub(r"\s+", "_", str(c)).strip().strip("\ufeff").lower() for c in df.columns]
    return df

# ---------- Read three input tables ----------
ops  = pd.read_csv(OPS_PATH)
tech = read_table_robust(TECH_PATH)
poldf = read_table_robust(POLI_PATH)
pol  = poldf.iloc[0].to_dict()

# ---------- Utilities ----------
def pick_tech_row(tech_name: str) -> dict:
    """Selects the corresponding row from tech_params; S+S uses the 'Solar_PV_Storage' row."""
    if tech_name.startswith("Solar_PV_Storage"):
        return tech[tech["tech"]=="Solar_PV_Storage"].iloc[0].to_dict()
    else:
        return tech[tech["tech"]==tech_name].iloc[0].to_dict()

def parse_duration_hours(tech_name: str) -> int | None:
    """Parses duration in hours from the name, e.g., 'Solar_PV_Storage_4h' -> 4."""
    m = re.search(r"Storage_(\d+)h", tech_name)
    return int(m.group(1)) if m else None

def try_get_batt_discharge_mwh(zone: str, year: int, tech_name: str) -> float | None:
    """Reads out/battery_dispatch_{zone}_{year}_{dur}h.*, estimates annual discharge MWh; returns None if not found."""
    dur = parse_duration_hours(tech_name)
    if dur is None:
        return None
    pat = str(OUT_DIR / f"battery_dispatch_{zone}_{year}_{dur}h.*")
    cands = glob(pat)
    if not cands:
        return None
    fp = cands[0]
    df = pd.read_parquet(fp) if fp.endswith(".parquet") else pd.read_csv(fp)
    if "discharge_mw" not in df.columns:
        return None
    return float(df["discharge_mw"].sum())   # MW·h = MWh

def npv(cfs, r):
    return sum(cf/((1+r)**t) for t, cf in enumerate(cfs))

def irr(cfs, guess=0.08):
    """Simple Newton's method for IRR; can use numpy_financial.irr if unstable."""
    r = guess
    for _ in range(200):
        f  = sum(cf/((1+r)**t) for t, cf in enumerate(cfs))
        df = sum(-t*cf/((1+r)**(t+1)) for t, cf in enumerate(cfs))
        if abs(df) < 1e-12:
            break
        r_new = r - f/df
        if not np.isfinite(r_new):
            break
        if abs(r_new - r) < 1e-10:
            r = r_new
            break
        r = r_new
    return r


def capacity_mw_lookup(tech_name: str, zone: str, year: int) -> float | None:
    m = {
        ("Solar_PV",                 zone, year): 100.0,
        ("Solar_PV_Storage_4h",      zone, year): 100.0,
        ("Solar_PV_Storage_3h",      zone, year): 100.0,
        ("Solar_PV_Storage_2h",      zone, year): 100.0,
        # ("CCGT", zone, year): 500.0,
    }
    if tech_name.startswith("Solar_PV_Storage"):
        key = (f"Solar_PV_Storage_{parse_duration_hours(tech_name)}h", zone, year)
        return m.get(key, m.get(("Solar_PV_Storage_4h", zone, year)))
    return m.get((tech_name, zone, year))

# ---------- Main process: Generate 4.3/4.4 ----------
rows_cf, rows_sum = [], []

for _, o in ops.iterrows():
    tech_name  = o["tech"]
    zone       = str(o["zone"]).upper()
    year       = int(o["year"])

    annual_mwh = float(o["annual_mwh"])
    rev        = float(o["annual_revenue_usd"])
    batt_profit= float(o.get("annual_batt_profit_usd", 0.0))  # In S+S, revenue already includes battery profit or it's listed separately

    tref = pick_tech_row(tech_name)
    capex   = float(tref["capex_per_kw"])   # $/kW
    fom     = float(tref["fom_per_kw_yr"])  # $/kW-yr
    vom     = float(tref["vom_per_mwh"])    # $/MWh
    life    = int(tref["lifetime_years"])   # For YEARS
    wacc    = float(tref.get("wacc", tref.get("wacc_base", 0.08)))

    cap_mw = capacity_mw_lookup(tech_name, zone, year)
    if cap_mw is None or np.isnan(cap_mw):
        continue
    cap_kw = cap_mw * 1000.0

    capex_upfront_total = cap_kw * capex

    fixed_om_total     = cap_kw * fom
    variable_om_total  = annual_mwh * vom

    fuel_total, carbon_total = 0.0, 0.0
    if tech_name == "CCGT":
        gas_price     = float(pol.get("gas_price_usd_per_mmbtu", 4.0))
        heat_rate     = float(tref.get("heat_rate_mmbtu_per_mwh", 7.0))
        co2_ccgt      = float(pol.get("co2_t_per_mwh_ccgt", 0.36))
        carbon_price  = float(pol.get("carbon_price_usd_per_t", 0.0))
        fuel_total    = gas_price * heat_rate * annual_mwh
        carbon_total  = carbon_price * co2_ccgt * annual_mwh

    throughput_cost_total = 0.0
    if INCLUDE_THROUGHPUT_COST and tech_name.startswith("Solar_PV_Storage"):
        annual_batt_discharge_mwh = try_get_batt_discharge_mwh(zone, year, tech_name)
        if annual_batt_discharge_mwh and annual_batt_discharge_mwh > 0:
            throughput_cost_total = annual_batt_discharge_mwh * throughput_cost_per_mwh

    annual_cost = fixed_om_total + variable_om_total + fuel_total + carbon_total + throughput_cost_total
    annual_net  = rev - annual_cost

    YEARS = life  # Each technology uses its own lifetime

    # Cash flows (can include battery replacement)
    cfs = [-capex_upfront_total]
    for t in range(1, YEARS+1):
        extra_capex = 0.0
        if INCLUDE_BATTERY_REPLACEMENT and tech_name.startswith("Solar_PV_Storage") and t == battery_replacement_year:
            extra_capex = - cap_kw * capex * battery_replacement_ratio
        cfs.append(annual_net + extra_capex)

    irr_v = irr(cfs)
    npv_v = npv(cfs, wacc)   # Discount rate uses each technology's WACC (can be fixed if uniform discounting is desired)

    # Payback period: if annual_net <= 0 or > lifetime -> None
    if annual_net <= 0:
        payback = None
    else:
        pb = capex_upfront_total / annual_net
        payback = pb if pb <= YEARS else None

    rows_sum.append({
        "tech": tech_name, "zone": zone, "year": year,
        "npv_usd": round(npv_v, 0),
        "irr_pct": round(irr_v*100, 2),
        "payback_year": None if payback is None else round(payback, 1),
        "annual_net_usd": round(annual_net, 0),
        "capex_upfront_usd": round(capex_upfront_total, 0),
        "fixed_om_usd": round(fixed_om_total, 0),
        "variable_om_usd": round(variable_om_total, 0),
        "fuel_usd": round(fuel_total, 0),
        "carbon_usd": round(carbon_total, 0),
        "throughput_cost_usd": round(throughput_cost_total, 0),
    })

    # Amortization table (year by year)
    for t in range(0, YEARS+1):
        if t == 0:
            rows_cf.append({
                "tech": tech_name, "zone": zone, "year": year, "t": t,
                "capex_outflow": capex_upfront_total,
                "fixed_om": 0.0, "variable_om": 0.0,
                "fuel_cost": 0.0, "carbon_cost": 0.0,
                "throughput_cost": 0.0,
                "revenue": 0.0, "net_cashflow": -capex_upfront_total
            })
        else:
            extra_capex = 0.0
            if INCLUDE_BATTERY_REPLACEMENT and tech_name.startswith("Solar_PV_Storage") and t == battery_replacement_year:
                extra_capex = - cap_kw * capex * battery_replacement_ratio
            rows_cf.append({
                "tech": tech_name, "zone": zone, "year": year, "t": t,
                "capex_outflow": -extra_capex if extra_capex < 0 else 0.0,  # Display as positive CAPEX amount
                "fixed_om": fixed_om_total, "variable_om": variable_om_total,
                "fuel_cost": fuel_total, "carbon_cost": carbon_total,
                "throughput_cost": throughput_cost_total,
                "revenue": rev,
                "net_cashflow": annual_net + extra_capex
            })

# Export
cf_df  = pd.DataFrame(rows_cf)
sum_df = pd.DataFrame(rows_sum).drop_duplicates(subset=["tech","zone","year"])

cf_df.to_csv(OUT_DIR / "cashflows_project.csv", index=False)
sum_df.to_csv(OUT_DIR / "irr_npv_table.csv", index=False)
print("saved ->", OUT_DIR / "cashflows_project.csv", "and", OUT_DIR / "irr_npv_table.csv")

saved -> out/cashflows_project.csv and out/irr_npv_table.csv
